<a href="https://colab.research.google.com/github/Abdul-Maajith/Research-Google-Docs-Multi-Agent/blob/main/Autogen_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyautogen langchain langchain-community langchain-experimental duckduckgo-search praw stackapi arxiv wikipedia google-api-python-client pygithub python-socketio

In [ ]:
def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace (' ', '_'),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
      function_schema["parameters"]["properties"] = tool.args

    return function_schema

In [ ]:
import os
from google.colab import userdata
config_list = [
    {
        'model': 'gpt-4',
        'api_key': userdata.get('openAIKey')
    }
]

def is_termination_msg(content) -> bool:
    have_content = content.get("content", None) is not None
    if have_content and "TERMINATE" in content["content"]:
        return True
    return False

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def create_and_share_google_doc(data, document_name, email):
    # Load the credentials from the JSON file
    credentials = service_account.Credentials.from_service_account_file(
        '/content/creds.json',
        scopes=['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']
    )

    # Build the service for Google Docs
    docs_service = build('docs', 'v1', credentials=credentials)
    # Build the service for Google Drive
    drive_service = build('drive', 'v3', credentials=credentials)

    # Document ID of the document to update. If it doesn't exist, create a new one.
    document_id = 'your-document-id'

    try:
        # Try to get the document
        document = docs_service.documents().get(documentId=document_id).execute()
        print(f"Document '{document_name}' already exists.")
    except HttpError as error:
        # If the document doesn't exist, create a new one
        if error.resp.status == 404:
            document = docs_service.documents().create(body={
                'title': document_name
            }).execute()
            document_id = document.get('documentId')
            print(f"Document '{document_name}' created with ID: {document_id}")
        else:
            print(f"An error occurred: {error}")
            return

    # Update the document with the provided data
    requests = [
        {
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': data
            }
        }
    ]

    result = docs_service.documents().batchUpdate(
        documentId=document_id,
        body={'requests': requests}
    ).execute()

    print(f"Document '{document_name}' updated with custom data.")

    # Share the document with the specified email
    try:
        permission = {
            'type': 'user',
            'role': 'writer',
            'emailAddress': email
        }
        drive_service.permissions().create(
            fileId=document_id,
            body=permission,
            fields='id'
        ).execute()
        print(f"Document shared with {email}")
        return {
            f"Document '{document_name}' created with ID: {document_id}",
            f"Document '{document_name}' updated with custom data: {data}",
            f"Document saved onto Drive: {email}, please check shared folder on your GDrive"
        }
    except Exception as error:
        print(f"An error occurred while sharing the document: {error}")

In [ ]:
import requests
from bs4 import BeautifulSoup

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from bs4 import BeautifulSoup
from langchain.chat_models import ChatOpenAI
import json
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openAIKey')

def summary(content):
  llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo-16k-0613")

  text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size = 10000, chunk_overlap=500)
  docs = text_splitter.create_documents([content])

  map_prompt = """
    Write a summary of the following text:
    "{text}"
    SUMMARY:
    """
  map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

  summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt = map_prompt_template,
    combine_prompt = map_prompt_template,
    verbose = False
  )

  output = summary_chain.run(input_documents=docs)

  return output

# Function 1: WebScraping
def web_scraping(url: str):
  response = requests.get(
        url='https://app.scrapingbee.com/api/v1/',
        params={
            'api_key': userdata.get('scrapingBeeKey'),
            'url': url,
        },

    )
  soup = BeautifulSoup(response.content, "html.parser")
  text = soup.get_text()
  print(len(text))
  if len(text) > 10000:
    output = summary(text)
    return output
  else:
    return text

# Function 2 : search Keyword on Google SERP
def google_search(query):
  url = "https://google.serper.dev/search"

  payload = json.dumps({
    "q": query
  })

  headers = {
    'X-API-KEY': userdata.get('serper_api_key'),
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.json()


In [ ]:
llm_config={
    "functions": [
        {
            "name": "web_scraping",
            "description": "Does Webscraping on a given URL and returns the result in a txt format",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the website, which is to be scraped",
                    }
                },
                "required": ["url"],
            },
        },
        {
            "name": "google_search",
            "description": "It takes a search query as input and returns the search results obtained from Google using the SerpAPI",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to search the indexed Wikipedia knowledge base for relevant information.",
                    }
                },
                "required": ["query"],
            },
        },
        {
            "name": "create_and_share_google_doc",
            "description": "The function receives the name of a document file, its content, and an email address. It then creates a Google Doc with the specified name, writes the content into it, and stores it in the Google Drive associated with the provided email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {
                        "type": "string",
                        "description": "The data which is to be written onto the google docs file",
                    },
                    "document_name": {
                        "type": "string",
                        "description": "The name of the document to be created or updated in Google Docs",
                    },
                    "email": {
                        "type": "string",
                        "description": "The email address to share the document with",
                    }
                },
                "required": ["data", "document_name", "email"],
            },
        },
    ],
    "seed": 42,
    "config_list": config_list,
    "temperature": 0
}

In [ ]:
import autogen

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=is_termination_msg,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    },
)

assistant = autogen.AssistantAgent(
    name="assistant",
    system_message='''Complete the task. Begin by utilising the `google_search` function to
    index the relevant webpage URL's, followed by using the `web_scraping`
    function to retrieve the information from the particular URL and finally use create_and_share_google_doc to create and share the google Docs to the corresponding email address
    Conclude with TERMINATE once the answer has been delivered.
    ''',
    llm_config=llm_config,
)

moderator = autogen.AssistantAgent(
    name="moderator",
    system_message='''Your role is to guide the `assistant` by suggesting what information may be
    necessary to effectively address the URL's.
    Ensure the assistant is making accurate and relevant URL's
    to each function to retrieve the needed information for task completion.
    Conclude with TERMINATE once the answer has been delivered.
    ''',
    llm_config=llm_config,
    # human_input_mode="NEVER"
)

user_proxy.register_function(
    function_map={
      "web_scraping": web_scraping,
      "google_search":google_search,
      "create_and_share_google_doc": create_and_share_google_doc
    }
)

groupchat = autogen.GroupChat(agents=[user_proxy, moderator, assistant], messages=[], max_round=20)
llm_config_without_functions = {key: value for key, value in llm_config.items() if key != 'functions'}
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_without_functions)
user_proxy.initiate_chat(manager, message="Create a google Docs on the topic: Recent Updates on AI Agents, and share it with this email: abdulmaajith0703@gmail.com")

user_proxy (to chat_manager):

Create a google Docs on the topic: Recent Updates on AI Agents, and share it with this email: abdulmaajith0703@gmail.com

--------------------------------------------------------------------------------


assistant (to chat_manager):

Sure, let's start by searching for recent updates on AI agents.
***** Suggested function call: google_search *****
Arguments: 
{
  "query": "Recent Updates on AI Agents"
}
**************************************************

--------------------------------------------------------------------------------



>>>>>>>> EXECUTING FUNCTION google_search...


user_proxy (to chat_manager):

***** Response from calling function (google_search) *****
{'searchParameters': {'q': 'Recent Updates on AI Agents', 'type': 'search', 'engine': 'google'}, 'organic': [{'title': 'Forget Chatbots. AI Agents Are the Future - WIRED', 'link': 'https://www.wired.com/story/fast-forward-forget-chatbots-ai-agents-are-the-future/', 'snippet': 'Startups and tech giants are trying to move from chatbots that offer help via text, to AI agents that can get stuff done. Recent demos ...', 'date': 'Mar 14, 2024', 'attributes': {'Missing': 'Updates | Show results with:Updates'}, 'position': 1}, {'title': "Tech Companies Bet the World is Ready for 'AI Agents' - Bloomberg", 'link': 'https://www.bloomberg.com/news/newsletters/2024-02-15/tech-companies-bet-the-world-is-ready-for-ai-agents', 'snippet': 'A number of startups, including Adept, Imbue and Luda, have raised millions in funding to pursue agents, a technology dubbed agentic AI. This ...', 'date': 'Feb 15, 2024', 'posi

moderator (to chat_manager):

The search results provide several articles and sources that discuss recent updates on AI agents. To create a comprehensive document, we should scrape information from these sources. Let's start with the first three links:

1. "Forget Chatbots. AI Agents Are the Future - WIRED" - https://www.wired.com/story/fast-forward-forget-chatbots-ai-agents-are-the-future/
2. "Tech Companies Bet the World is Ready for 'AI Agents' - Bloomberg" - https://www.bloomberg.com/news/newsletters/2024-02-15/tech-companies-bet-the-world-is-ready-for-ai-agents
3. "Artificial Intelligence News - ScienceDaily" - https://www.sciencedaily.com/news/computers_math/artificial_intelligence/

We will use the web_scraping function to extract information from these sources.
***** Suggested function call: web_scraping *****
Arguments: 
{
  "url": "https://www.wired.com/story/fast-forward-forget-chatbots-ai-agents-are-the-future/"
}
*************************************************

---------

...


>>>>>>>> EXECUTING FUNCTION create_and_share_google_doc...
Document 'Recent Updates on AI Agents' created with ID: 1B98kg8T0X3RG7WmhjxQ7FZLf2m12ko8Ce9GmTOFQFZ8
Document 'Recent Updates on AI Agents' updated with custom data.


Document shared with abdulmaajith0703@gmail.com
user_proxy (to chat_manager):

***** Response from calling function (create_and_share_google_doc) *****
{'Document \'Recent Updates on AI Agents\' updated with custom data: Based on the information obtained from the web scraping, here is a summary of the recent updates on AI agents:\n\n1. From the Wired article, we learn about the shift from chatbots to AI agents. Startups and tech giants are moving from chatbots that offer help via text, to AI agents that can perform tasks. A startup called Cognition AI released a demo showing an AI program called Devin performing work usually done by software engineers. Devin is branded as an "AI software developer" and can plan how to solve a problem, write the code, and then test and implement it. Google DeepMind has also developed an agent called SIMA, which plays video games and can perform over 600 tasks. \n\n2. The Bloomberg article mentions that 2023 was the year of the AI chatbot, and 2024 is sh

TERMINATE


moderator (to chat_manager):

The Google Doc titled "Recent Updates on AI Agents" has been created and shared with the email address abdulmaajith0703@gmail.com. Please check the shared folder on your Google Drive to access the document. 

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Create a google Docs on the topic: Recent Updates on AI Agents, and share it with this email: abdulmaajith0703@gmail.com', 'role': 'assistant'}, {'content': "Sure, let's start by searching for recent updates on AI agents.", 'function_call': {'arguments': '{\n  "query": "Recent Updates on AI Agents"\n}', 'name': 'google_search'}, 'name': 'assistant', 'role': 'assistant'}, {'content': '{\'searchParameters\': {\'q\': \'Recent Updates on AI Agents\', \'type\': \'search\', \'engine\': \'google\'}, \'organic\': [{\'title\': \'Forget Chatbots. AI Agents Are the Future - WIRED\', \'link\': \'https://www.wired.com/story/fast-forward-forget-chatbots-ai-agents-are-the-future/\', \'snippet\': \'Startups and tech giants are trying to move from chatbots that offer help via text, to AI agents that can get stuff done. Recent demos ...\', \'date\': \'Mar 14, 2024\', \'attributes\': {\'Missing\': \'Updates | Show results with:Updates\'}, \'position\': 